10.01.23

# Select slices that experts will annotate

*IDEA*: select a total of 10 slices
- 3 where the model perform poorly
    - if it is obvious why -> discard
    - if it is interesting -> keep
- 7 selected "randomly" among the top k frames with the most pixels annotated

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
#import glob
import os
import imageio
import napari
import matplotlib.pyplot as plt
import math
import seaborn as sns
import pandas as pd

#from skimage.io import imsave

from in_out_tools import load_movies_ids, load_annotations_ids
from visualization_tools import get_discrete_cmap, get_labels_cmap, get_annotations_contour
from metrics_tools import compute_iou, empty_marginal_frames
from data_processing_tools import get_processed_result

## Set global parameters

In [3]:
BASEDIR = os.path.abspath('')
BASEDIR

'c:\\Users\\dotti\\sparks_project\\sparks'

In [4]:
# configure Napari cmap
cmap = get_discrete_cmap(name='gray', lut=16)
labels_cmap = get_labels_cmap()

# frames ignored by loss fct during training
ignore_frames_loss = 6

In [5]:
classes = ['sparks', 'puffs', 'waves']

class_to_nb = {'sparks': 1,
               'puffs': 3,
               'waves': 2,
               'ignore_rois': 4
              }

movie_ids = ["05","10","15","20","25","32","34","40","45"]
#movie_ids = ["05","34"]

In [6]:
dataset_dir = os.path.join("..", "data", "sparks_dataset")

## Load movies, annotations & predictions

In [7]:
# Config model to load
training_name = 'TEMP_new_annotated_peaks_physio'
epoch = 100000

preds_dir = os.path.join("trainings_validation", training_name)

In [8]:
### Load movies
xs = load_movies_ids(data_folder=dataset_dir,
                     ids=movie_ids,
                     names_available=True,
                     movie_names="video")

### Load annotations
ys = load_annotations_ids(data_folder=dataset_dir,
                          ids=movie_ids,
                          mask_names="class_label")

### Load predictions
sparks_filenames = {movie_id: os.path.join(preds_dir, f"{training_name}_{epoch}_{movie_id}_sparks.tif") for movie_id in movie_ids}
puffs_filenames = {movie_id: os.path.join(preds_dir, f"{training_name}_{epoch}_{movie_id}_puffs.tif") for movie_id in movie_ids}
waves_filenames = {movie_id: os.path.join(preds_dir, f"{training_name}_{epoch}_{movie_id}_waves.tif") for movie_id in movie_ids}

sparks = {movie_id: np.asarray(imageio.volread(f)) for movie_id, f in sparks_filenames.items()}
puffs = {movie_id: np.asarray(imageio.volread(f)) for movie_id, f in puffs_filenames.items()}
waves = {movie_id: np.asarray(imageio.volread(f)) for movie_id, f in waves_filenames.items()}

TiffPage 0: TypeError: read_bytes() missing 3 required positional arguments: 'dtype', 'count', and 'offsetsize'


### Compute class weights

To use in order to get frames with the most annotated pixels in a weighted way

In [13]:
# define function to compute class weights
def compute_class_weights(dataset, w0=1, w1=1, w2=1, w3=1):
    '''
    Modified version of 'compute_class_weights' in  training_inference_tools.py
    (using numpy arrays instead of a SparkDataset instance)

    dataset: list of numpy arrays representing annotation masks with values
             between 0 and 4
    '''

    # For 4 classes
    count0 = 0
    count1 = 0
    count2 = 0
    count3 = 0

    for y in dataset:
        count0 += np.count_nonzero(y == 0)
        count1 += np.count_nonzero(y == 1)
        count2 += np.count_nonzero(y == 2)
        count3 += np.count_nonzero(y == 3)

    total = count0 + count1 + count2 + count3

    w0_new = w0 * total / (4 * count0) if count0 != 0 else 0
    w1_new = w1 * total / (4 * count1) if count1 != 0 else 0
    w2_new = w2 * total / (4 * count2) if count2 != 0 else 0
    w3_new = w3 * total / (4 * count3) if count3 != 0 else 0

    weights = [w0_new, w1_new, w2_new, w3_new]
    return weights

In [14]:
# compute class weights
w_bg, w_sparks, w_waves, w_puffs = compute_class_weights(dataset=list(ys.values()))

# define dict with weights
w_dict = {'sparks': w_sparks,
          'puffs': w_puffs,
          'waves': w_waves}

In [15]:
w_dict

{'sparks': 261.3059303936147,
 'puffs': 34.31009686297766,
 'waves': 12.561785793660091}

### Process predictions to obtain segmentation masks with small objects removed

In [16]:
# parameters necessary to process predictions

# physiological params (for spark peaks results)
pixel_size = 0.2 # 1 pixel = 0.2 um x 0.2 um
min_dist_xy = round(1.8 / pixel_size) # min distance in space
time_frame = 6.8 # 1 frame = 6.8 ms
min_dist_t = round(20 / time_frame) # min distance in time

# spark instances detection parameters
min_dist_xy = min_dist_xy
min_dist_t = min_dist_t
radius = math.ceil(min_dist_xy / 2)
y, x = np.ogrid[-radius : radius + 1, -radius : radius + 1]
disk = x**2 + y**2 <= radius**2
conn_mask = np.stack([disk] * (min_dist_t), axis=0)

# connectivity for event instances detection
connectivity = 26
sigma = 3

# TODO: use better parameters !!!
pixel_size = 0.2
spark_min_width = 3
spark_min_t = 3
puff_min_t = 5
wave_min_width = round(15 / pixel_size)

# maximal gap between two predicted puffs or waves that belong together
max_gap = 2  # i.e., 2 empty frames

In [17]:
preds_segmentation = {}

for movie_id in movie_ids:
    _, preds_segmentation[movie_id], _ = get_processed_result(
            sparks=sparks[movie_id],
            puffs=puffs[movie_id],
            waves=waves[movie_id],
            xs=xs[movie_id],
            conn_mask=conn_mask,
            connectivity=connectivity,
            max_gap=max_gap,
            sigma=sigma,
            wave_min_width=wave_min_width,
            puff_min_t=puff_min_t,
            spark_min_t=spark_min_t,
            spark_min_width=spark_min_width,
        )

	Events detection threshold: 0.7783508
	Number of sparks detected by nonmaxima suppression: 148
	Events detection threshold: 0.77862227
	Number of sparks detected by nonmaxima suppression: 40
	Not all sparks were labelled, computing missing events...
	Number of sparks before correction: 40
	Number of sparks after correction: 41
	Events detection threshold: 0.7812276
	Number of sparks detected by nonmaxima suppression: 22
	Events detection threshold: 0.78007615
	Number of sparks detected by nonmaxima suppression: 96
	Events detection threshold: 0.77644074
	Number of sparks detected by nonmaxima suppression: 102
	Events detection threshold: 0.76853275
	Number of sparks detected by nonmaxima suppression: 8
	Events detection threshold: 0.7502694
	Number of sparks detected by nonmaxima suppression: 47
	Events detection threshold: 0.72635245
	Number of sparks detected by nonmaxima suppression: 4
	Events detection threshold: 0.72820926
	Number of sparks detected by nonmaxima suppression: 40
	

## Compute statistics

In [18]:
# configure dataframe
cols = ['movie ID', 'frame ID', 
        'sparks IoU', 'puffs IoU', 'waves IoU', 'average IoU',
        'sparks pixels', 'puffs pixels', 'waves pixels', 
        'sparks weighted pixels', 'puffs weighted pixels', 
        'waves weighted pixels', 'weighted sum pixels']

frames_stats_df = pd.DataFrame(columns=cols)

# iterate over movies and frame to compute statistics
for movie_id in movie_ids:
    # remove frames ignored by loss function
    sample_ys = empty_marginal_frames(video=ys[movie_id],
                                      n_frames=ignore_frames_loss)
    
    sample_preds = {} # where predicted segmentation will be stored
    for event_type in classes:
        sample_preds[event_type] = empty_marginal_frames(
                                    video=preds_segmentation[movie_id][event_type],
                                    n_frames=ignore_frames_loss
                                    )

    # mask ignored during training
    ignore_mask = sample_ys==class_to_nb['ignore_rois']

    # sample_preds is a dict with keys 'sparks','puffs' and 'waves'
    for frame_id in range(len(sample_ys)):
        # create frame dict that will be appended to dataframe
        frame_dict = {'movie ID': movie_id,
                      'frame ID': frame_id,
                      'average IoU' : 0,
                      'weighted sum pixels' : 0}

        for event_type in classes:
            # get binary masks corrisponding to event type
            ys_frame_class = sample_ys[frame_id]==class_to_nb[event_type]
            preds_frame_class = sample_preds[event_type][frame_id]

            # compute Intersection over Union score
            iou = compute_iou(ys_roi=ys_frame_class,
                              preds_roi=preds_frame_class,
                              ignore_mask=ignore_mask[frame_id])
            frame_dict[event_type+' IoU'] = iou
            frame_dict['average IoU'] += iou

            # count number of (WEIGHTED) annotated pixels
            nonzero_pixels = np.count_nonzero(ys_frame_class)
            frame_dict[event_type+' pixels'] = nonzero_pixels
            frame_dict[event_type+' weighted pixels'] = w_dict[event_type]*nonzero_pixels
            frame_dict['weighted sum pixels'] += w_dict[event_type]*nonzero_pixels
            
        frame_dict['average IoU'] /= len(classes)
        
        # create df from frame dict
        frame_df = pd.DataFrame([frame_dict])
        frames_stats_df = pd.concat([frames_stats_df, frame_df])

In [19]:
frames_stats_df

,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks pixels,puffs pixels,waves pixels,sparks weighted pixels,puffs weighted pixels,waves weighted pixels,weighted sum pixels
0,05,0,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0
0,05,1,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0
0,05,2,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0
0,05,3,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0
0,05,4,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,45,995,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0
0,45,996,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0
0,45,997,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0
0,45,998,1.0,1.0,1.0,1.0,0,0,0,0.0,0.0,0.0,0.0


### Discard empty frames (if both ys and preds are empty)

Idea: if nb pixels in annotations == 0 and IoU avg == 1 ==> nb pixels in preds = 0

In [20]:
frames_stats_df = frames_stats_df.loc[(frames_stats_df['average IoU']!=1) 
                                        | (frames_stats_df['weighted sum pixels']!=0)]

In [21]:
frames_stats_df

,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks pixels,puffs pixels,waves pixels,sparks weighted pixels,puffs weighted pixels,waves weighted pixels,weighted sum pixels
0,05,6,0.163488,0.0,0.0,0.054496,68,0,0,17768.803267,0.0,0.0,17768.803267
0,05,7,0.215539,0.0,0.0,0.071846,109,0,0,28482.346413,0.0,0.0,28482.346413
0,05,8,0.235012,0.0,0.0,0.078337,120,0,0,31356.711647,0.0,0.0,31356.711647
0,05,9,0.246341,0.0,1.0,0.415447,115,0,0,30050.181995,0.0,0.0,30050.181995
0,05,10,0.270270,0.0,1.0,0.423423,129,0,0,33708.465021,0.0,0.0,33708.465021
...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,45,989,1.000000,1.0,0.0,0.666667,0,0,0,0.000000,0.0,0.0,0.000000
0,45,990,0.000000,1.0,0.0,0.333333,0,0,0,0.000000,0.0,0.0,0.000000
0,45,991,0.000000,1.0,0.0,0.333333,0,0,0,0.000000,0.0,0.0,0.000000
0,45,992,0.000000,1.0,0.0,0.333333,0,0,0,0.000000,0.0,0.0,0.000000


## Select frames according to some criteria and visualize them in Napari

### Get frames where waves IoU is worst, and annotated waves aren't empty

In [51]:
# Get df of frames containing waves
waves_df = frames_stats_df.loc[frames_stats_df['waves pixels']!=0]

In [52]:
# Order waves df from worst waves IoU to best
waves_df.sort_values(['waves IoU', 'waves pixels'], ascending=[True, False]).head(10)

,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks pixels,puffs pixels,waves pixels,sparks weighted pixels,puffs weighted pixels,waves weighted pixels,weighted sum pixels
0,34,557,0.0,0.0,0.0,0.0,0,0,5500,0.0,0.0,69089.821865,69089.821865
0,34,556,0.0,0.0,0.0,0.0,0,0,5470,0.0,0.0,68712.968291,68712.968291
0,34,555,0.0,0.0,0.0,0.0,0,0,5402,0.0,0.0,67858.766857,67858.766857
0,34,558,0.0,0.0,0.0,0.0,0,0,5307,0.0,0.0,66665.397207,66665.397207
0,34,554,0.0,0.0,0.0,0.0,0,0,5273,0.0,0.0,66238.296490,66238.296490
0,34,548,0.0,0.0,0.0,0.0,0,0,5241,0.0,0.0,65836.319345,65836.319345
0,34,549,0.0,0.0,0.0,0.0,0,0,5231,0.0,0.0,65710.701487,65710.701487
0,34,559,0.0,0.0,0.0,0.0,0,0,5183,0.0,0.0,65107.735769,65107.735769
0,34,550,0.0,0.0,0.0,0.0,0,0,5181,0.0,0.0,65082.612197,65082.612197
0,34,553,0.0,0.0,0.0,0.0,0,0,5103,0.0,0.0,64102.792905,64102.792905


### Get frames where puffs IoU is worst, and annotated puffs aren't empty

In [25]:
# Get df of frames containing puffs
puffs_df = frames_stats_df.loc[frames_stats_df['puffs pixels']!=0]

In [45]:
# Order puffs df from worst puffs IoU to best
puffs_df.sort_values(['puffs IoU', 'puffs pixels'], ascending=[True, False]).head(10)

,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks pixels,puffs pixels,waves pixels,sparks weighted pixels,puffs weighted pixels,waves weighted pixels,weighted sum pixels
0,20,443,0.000000,0.0,1.0,0.333333,0,1088,0,0.000000,37329.385387,0.0,37329.385387
0,20,444,0.000000,0.0,1.0,0.333333,0,1082,0,0.000000,37123.524806,0.0,37123.524806
0,20,452,0.000000,0.0,1.0,0.333333,0,1080,0,0.000000,37054.904612,0.0,37054.904612
0,20,435,0.000000,0.0,1.0,0.333333,60,1062,0,15678.355824,36437.322868,0.0,52115.678692
0,20,431,0.023451,0.0,1.0,0.341150,84,1049,0,21949.698153,35991.291609,0.0,57940.989762
0,20,454,0.039216,0.0,1.0,0.346405,22,1018,0,5748.730469,34927.678607,0.0,40676.409075
0,20,448,0.000000,0.0,1.0,0.333333,0,1006,0,0.000000,34515.957444,0.0,34515.957444
0,20,445,0.000000,0.0,1.0,0.333333,0,998,0,0.000000,34241.476669,0.0,34241.476669
0,20,446,0.000000,0.0,1.0,0.333333,0,977,0,0.000000,33520.964635,0.0,33520.964635
0,20,429,0.029781,0.0,1.0,0.343260,86,953,0,22472.310014,32697.522310,0.0,55169.832324


### Get frames where sparks IoU is worst, and annotated sparks aren't empty

In [42]:
# Get df of frames containing sparks
sparks_df = frames_stats_df.loc[frames_stats_df['sparks pixels']!=0]

In [44]:
# Order sparks df from worst sparks IoU to best
sparks_df.sort_values(['sparks IoU', 'sparks pixels'], ascending=[True, False]).head(10)

,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks pixels,puffs pixels,waves pixels,sparks weighted pixels,puffs weighted pixels,waves weighted pixels,weighted sum pixels
0,15,54,0.0,0.359585,1.0,0.453195,507,910,0,132482.106710,31222.188145,0.0,163704.294855
0,15,63,0.0,0.451220,1.0,0.483740,463,759,0,120984.645772,26041.363519,0.0,147026.009291
0,15,56,0.0,0.405702,1.0,0.468567,450,832,0,117587.668677,28546.000590,0.0,146133.669267
0,15,59,0.0,0.390418,1.0,0.463473,424,902,0,110793.714487,30947.707370,0.0,141741.421857
0,15,61,0.0,0.393057,1.0,0.464352,424,796,0,110793.714487,27310.837103,0.0,138104.551590
0,15,58,0.0,0.391650,1.0,0.463883,418,931,0,109225.878905,31942.700179,0.0,141168.579084
0,15,60,0.0,0.399556,1.0,0.466519,408,805,0,106612.819601,27619.627975,0.0,134232.447575
0,15,55,0.0,0.421053,1.0,0.473684,406,788,0,106090.207740,27036.356328,0.0,133126.564068
0,15,57,0.0,0.400433,1.0,0.466811,401,837,0,104783.678088,28717.551074,0.0,133501.229162
0,15,62,0.0,0.455791,1.0,0.485264,384,732,0,100341.477271,25114.990904,0.0,125456.468175


### Get frames where average IoU is worst, and annotation aren't empty

In [53]:
# Get df of frames containing any events
events_df = frames_stats_df.loc[frames_stats_df['weighted sum pixels']!=0]

In [54]:
# Order events df from worst average IoU to best
sparks_df.sort_values(['average IoU', 'weighted sum pixels'], ascending=[True, False]).head(10)

,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks pixels,puffs pixels,waves pixels,sparks weighted pixels,puffs weighted pixels,waves weighted pixels,weighted sum pixels
0,34,583,0.0,0.0,0.0,0.0,101,0,4601,26391.898970,0.0,57796.776437,84188.675406
0,34,590,0.0,0.0,0.0,0.0,114,0,4216,29788.876065,0.0,52960.488906,82749.364971
0,34,591,0.0,0.0,0.0,0.0,107,0,4122,27959.734552,0.0,51779.681041,79739.415594
0,34,582,0.0,0.0,0.0,0.0,80,0,4495,20904.474431,0.0,56465.227143,77369.701574
0,34,597,0.0,0.0,0.0,0.0,63,0,4730,16462.273615,0.0,59417.246804,75879.520419
0,34,592,0.0,0.0,0.0,0.0,88,0,4128,22994.921875,0.0,51855.051756,74849.973631
0,34,598,0.0,0.0,0.0,0.0,54,0,4816,14110.520241,0.0,60497.560382,74608.080624
0,34,596,0.0,0.0,0.0,0.0,62,0,4615,16200.967684,0.0,57972.641438,74173.609122
0,34,595,0.0,0.0,0.0,0.0,68,0,4432,17768.803267,0.0,55673.834638,73442.637904
0,34,599,0.0,0.0,0.0,0.0,45,0,4871,11758.766868,0.0,61188.458601,72947.225469


### For each movie, get frames with most weighted annotated pixels

In [29]:
for movie_id in movie_ids:
    most_annotated_df = frames_stats_df.loc[frames_stats_df['movie ID']==movie_id]
    display(most_annotated_df.sort_values('weighted sum pixels', ascending=False).head(5))

,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks pixels,puffs pixels,waves pixels,sparks weighted pixels,puffs weighted pixels,waves weighted pixels,weighted sum pixels
0,05,66,0.247492,0.080046,1.0,0.442513,371,101,0,96944.500176,3465.319783,0.0,100409.819959
0,05,64,0.274427,0.000000,1.0,0.424809,379,28,0,99034.947619,960.682712,0.0,99995.630331
0,05,65,0.260251,0.000000,1.0,0.420084,376,36,0,98251.029828,1235.163487,0.0,99486.193315
0,05,67,0.222028,0.078920,1.0,0.433649,354,136,0,92502.299359,4666.173173,0.0,97168.472533
0,05,69,0.204163,0.094050,1.0,0.432737,344,175,0,89889.240055,6004.266951,0.0,95893.507006


,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks pixels,puffs pixels,waves pixels,sparks weighted pixels,puffs weighted pixels,waves weighted pixels,weighted sum pixels
0,10,409,0.0,0.189573,1.0,0.396524,193,3165,0,50432.044566,108591.456571,0.0,159023.501137
0,10,406,0.0,0.193106,1.0,0.397702,148,3133,0,38673.277698,107493.533472,0.0,146166.811170
0,10,492,0.0,0.456250,1.0,0.485417,0,4225,0,0.000000,144960.159246,0.0,144960.159246
0,10,485,1.0,0.378199,1.0,0.792733,0,4128,0,0.000000,141632.079850,0.0,141632.079850
0,10,474,0.0,0.314862,1.0,0.438287,0,4102,0,0.000000,140740.017332,0.0,140740.017332


,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks pixels,puffs pixels,waves pixels,sparks weighted pixels,puffs weighted pixels,waves weighted pixels,weighted sum pixels
0,15,54,0.000000,0.359585,1.0,0.453195,507,910,0,132482.106710,31222.188145,0.0,163704.294855
0,15,50,0.190476,0.529061,1.0,0.573179,525,671,0,137185.613457,23022.074995,0.0,160207.688452
0,15,46,0.282692,0.673621,1.0,0.652105,520,671,0,135879.083805,23022.074995,0.0,158901.158800
0,15,42,0.273267,0.716049,1.0,0.663106,505,643,0,131959.494849,22061.392283,0.0,154020.887132
0,15,29,0.406639,0.602439,1.0,0.669693,481,792,0,125688.152519,27173.596715,0.0,152861.749235


,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks pixels,puffs pixels,waves pixels,sparks weighted pixels,puffs weighted pixels,waves weighted pixels,weighted sum pixels
0,20,281,0.679505,0.541028,1.0,0.740178,614,1710,0,160441.841262,58670.265636,0.0,219112.106897
0,20,280,0.696581,0.532213,1.0,0.742931,604,1646,0,157828.781958,56474.419436,0.0,214303.201394
0,20,279,0.732036,0.530601,1.0,0.754212,589,1661,0,153909.193002,56989.070889,0.0,210898.263891
0,20,282,0.631793,0.561384,1.0,0.731059,581,1679,0,151818.745559,57606.652633,0.0,209425.398192
0,20,277,0.684515,0.516201,1.0,0.733572,572,1703,0,149466.992185,58430.094958,0.0,207897.087143


,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks pixels,puffs pixels,waves pixels,sparks weighted pixels,puffs weighted pixels,waves weighted pixels,weighted sum pixels
0,25,114,0.612245,0.346535,1.0,0.652927,240,57,0,62713.423294,1955.675521,0.0,64669.098816
0,25,111,0.613577,0.284091,1.0,0.632556,235,38,0,61406.893642,1303.783681,0.0,62710.677323
0,25,112,0.585859,0.279570,1.0,0.621809,232,42,0,60622.975851,1441.024068,0.0,62063.999920
0,25,113,0.584810,0.292135,1.0,0.625648,231,36,0,60361.669921,1235.163487,0.0,61596.833408
0,25,115,0.580214,0.348214,1.0,0.642809,217,71,0,56703.386895,2436.016877,0.0,59139.403773


,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks pixels,puffs pixels,waves pixels,sparks weighted pixels,puffs weighted pixels,waves weighted pixels,weighted sum pixels
0,32,791,0.632768,1.0,1.0,0.877589,224,0,0,58532.528408,0.0,0.0,58532.528408
0,32,790,0.626781,1.0,1.0,0.875594,220,0,0,57487.304687,0.0,0.0,57487.304687
0,32,792,0.601124,1.0,1.0,0.867041,214,0,0,55919.469104,0.0,0.0,55919.469104
0,32,794,0.613372,1.0,1.0,0.871124,211,0,0,55135.551313,0.0,0.0,55135.551313
0,32,789,0.604046,1.0,1.0,0.868015,209,0,0,54612.939452,0.0,0.0,54612.939452


,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks pixels,puffs pixels,waves pixels,sparks weighted pixels,puffs weighted pixels,waves weighted pixels,weighted sum pixels
0,34,682,0.742718,0.459356,0.0,0.400691,165,1297,3987,43115.478515,44500.195631,50083.839959,137699.514106
0,34,681,0.729064,0.460177,0.0,0.396414,160,1294,4018,41808.948863,44397.265341,50473.255319,136679.469523
0,34,683,0.739130,0.460860,0.0,0.399997,165,1300,3844,43115.478515,44603.125922,48287.504591,136006.109028
0,34,680,0.706806,0.462710,0.0,0.389839,146,1289,4064,38150.665837,44225.714856,51051.097465,133427.478159
0,34,684,0.765306,0.461954,0.0,0.409087,155,1292,3663,40502.419211,44328.645147,46013.821362,130844.885720


,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks pixels,puffs pixels,waves pixels,sparks weighted pixels,puffs weighted pixels,waves weighted pixels,weighted sum pixels
0,40,952,0.0,0.0,0.277504,0.092501,81,0,3045,21165.780362,0.0,38250.637742,59416.418104
0,40,950,0.0,0.0,0.288141,0.096047,77,0,3120,20120.556640,0.0,39192.771676,59313.328317
0,40,953,0.0,0.0,0.269863,0.089954,83,0,2983,21688.392223,0.0,37471.807022,59160.199245
0,40,951,0.0,0.0,0.286796,0.095599,78,0,3037,20381.862571,0.0,38150.143455,58532.006026
0,40,954,0.0,0.0,0.260563,0.086854,79,0,2982,20643.168501,0.0,37459.245237,58102.413738


,movie ID,frame ID,sparks IoU,puffs IoU,waves IoU,average IoU,sparks pixels,puffs pixels,waves pixels,sparks weighted pixels,puffs weighted pixels,waves weighted pixels,weighted sum pixels
0,45,653,1.0,1.0,0.604427,0.868142,0,0,9802,0.0,0.0,123130.624349,123130.624349
0,45,652,1.0,1.0,0.608747,0.869582,0,0,9799,0.0,0.0,123092.938992,123092.938992
0,45,654,1.0,1.0,0.599400,0.866467,0,0,9790,0.0,0.0,122979.882920,122979.882920
0,45,651,1.0,1.0,0.611623,0.870541,0,0,9767,0.0,0.0,122690.961847,122690.961847
0,45,656,1.0,1.0,0.588595,0.862865,0,0,9733,0.0,0.0,122263.861130,122263.861130


### Visualize sample movie with Napari

In [71]:
movie_id = '45'

# get predicted segmentation with values between 0 and 3
preds = np.zeros_like(ys[movie_id])
for event_type in classes:
    preds += class_to_nb[event_type] * preds_segmentation[movie_id][event_type]

# get contours of annotated mask, for visualization
ys_contours = get_annotations_contour(annotations=ys[movie_id], contour_val=2)

In [72]:

viewer = napari.Viewer()
viewer.add_image(xs[movie_id], 
                 name='original movie', 
                 colormap=('colors',cmap)
                )

viewer.add_labels(preds,
                  name='predicted segmentation',
                  opacity=0.5,
                  color=labels_cmap
                 )

viewer.add_labels(ys_contours,
                  name='annotated segmentation',
                  opacity=0.8,
                  color=labels_cmap
                 )

<Labels layer 'annotated segmentation' at 0x29c6b69e850>